import modules

In [31]:
import pandas as pd
import os
current_directory = os.getcwd()
print("현재 디렉토리:", current_directory)

현재 디렉토리: d:\myhub\Project1\mid_code


load data

In [32]:
cny = pd.read_csv('../dataset/CNY_KRW.csv')[['날짜','종가','시가']].rename(
    columns={'날짜':'date',
             '종가':'close',
             '시가':'open'})

In [33]:
cny.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 66 entries, 0 to 65
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   date    66 non-null     object 
 1   close   66 non-null     float64
 2   open    66 non-null     float64
dtypes: float64(2), object(1)
memory usage: 1.7+ KB


#### 환율 변동값 전처리
* 전일 변동율 = (전날종가-전날시가)/전날시가 yesterday_change <br>
* 야간 변동율 = (당일시가-전날종가)/전날종가 night_change <br>
* 데이터 기간 : 2023/07/31 ~ 2023/10/20

In [34]:
# 파생변수 생성 (전일 변동율 = yesterday_change)
cny['cny_yesterday_change'] = round((cny['close']-cny['open'])/cny['open']*100, 2)
cny['cny_yesterday_change'] = cny['cny_yesterday_change'].shift(1)

# 파생변수 생성(야간 변동율 = night_change)
cny['close_shift'] = cny['close'].shift(1)
cny['cny_night_change'] = round((cny['open']-cny['close_shift'])/cny['close_shift']*100,2)

cny.drop('close_shift',axis=1,inplace=True)

In [35]:
# 데이터 기간 : 2023/07/31 ~ 2023/10/20
# cny = cny.loc[cny['date']<='2023-10-20',].loc[cny['date']>='2023-07-31',]
cny = cny[['date','cny_yesterday_change','cny_night_change']]

In [36]:
# 날짜순으로 오름차순 정렬
cny.sort_values(by='date', inplace=True)
cny.reset_index(drop=True, inplace=True)

In [37]:
print(cny.head(), cny.tail())

           date  cny_yesterday_change  cny_night_change
0  2023- 07- 28                  0.45              0.06
1  2023- 07- 31                  0.35             -0.82
2  2023- 08- 01                  0.53             -0.91
3  2023- 08- 02                  0.30             -0.88
4  2023- 08- 03                  0.25             -0.64             date  cny_yesterday_change  cny_night_change
61  2023- 10- 23                  0.09              0.41
62  2023- 10- 24                  0.49             -0.62
63  2023- 10- 25                 -0.01             -0.53
64  2023- 10- 26                  0.14             -0.05
65  2023- 10- 27                   NaN               NaN


In [38]:
cny.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 66 entries, 0 to 65
Data columns (total 3 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   date                  66 non-null     object 
 1   cny_yesterday_change  65 non-null     float64
 2   cny_night_change      65 non-null     float64
dtypes: float64(2), object(1)
memory usage: 1.7+ KB


기간 통일

In [39]:
ks200 = pd.read_csv('../dataset/clean_feature/ks200_present.csv', index_col=0)
# 데이터 타입 변경
cny['date'] = pd.to_datetime(cny['date'])
ks200.date = pd.to_datetime(ks200.date)
cny = pd.merge(cny, ks200, left_on='date', right_on='date', how='inner')[cny.columns]

In [40]:
print(cny.head(), cny.tail())

        date  cny_yesterday_change  cny_night_change
0 2023-07-31                  0.35             -0.82
1 2023-08-01                  0.53             -0.91
2 2023-08-02                  0.30             -0.88
3 2023-08-03                  0.25             -0.64
4 2023-08-04                 -0.09             -0.18          date  cny_yesterday_change  cny_night_change
49 2023-10-16                  0.30              0.14
50 2023-10-17                 -0.02             -0.35
51 2023-10-18                  0.05             -0.16
52 2023-10-19                 -0.65              0.50
53 2023-10-20                 -0.61              1.16


In [41]:
cny.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 54 entries, 0 to 53
Data columns (total 3 columns):
 #   Column                Non-Null Count  Dtype         
---  ------                --------------  -----         
 0   date                  54 non-null     datetime64[ns]
 1   cny_yesterday_change  54 non-null     float64       
 2   cny_night_change      54 non-null     float64       
dtypes: datetime64[ns](1), float64(2)
memory usage: 1.4 KB


csv 파일로 새로 저장

In [42]:
cny.to_csv("../dataset/clean_feature/cny_krw_final.csv")